## Cross-city evaluation
#### Leave-one-city-out
#### Leave-one-city-out-within-country
#### All-to-all city prediction
#### Meta learner

In [ ]:
from helper import get_training_data, get_csv_as_gpd, get_best_pca_lasso_model, get_best_lasso_model, train_xgboost
from sklearn.preprocessing import StandardScaler, RobustScaler
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import random
from scipy import stats

%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

## Generate master data set with all cities

In [3]:
cities_fr = ['marseille', 'lyon', 'paris']
cities_de = ['berlin', 'hamburg', 'bremen']

agg_full = pd.DataFrame()

if (len(cities_fr) !=0):
    for city in cities_fr:
        agg = get_training_data(city, 'FR', 1000, 'count', 2015)
        agg_full = agg_full.append(agg)
        
if (len(cities_de) !=0):
    for city in cities_de:
        agg = get_training_data(city, 'DE', 1000, 'count', 2015)
        agg_full = agg_full.append(agg)
    
print(agg_full.shape)
agg_full=agg_full.dropna(axis = 1)
agg_full = agg_full.reset_index(drop = True)
print(agg_full.shape)
agg_full

shape of training data (344, 243)
shape of training data (176, 243)
shape of training data (861, 243)
shape of training data (122, 243)
shape of training data (125, 243)
shape of training data (129, 243)
(1757, 243)
(1757, 243)


,assigned_city,districts_admin_level_11_id,convenience_area_count_1000,camera_surveillance_area_count_1000,tourist_info_area_count_1000,attraction_area_count_1000,pharmacy_area_count_1000,post_box_area_count_1000,bank_area_count_1000,bakery_area_count_1000,...,jewish_min_dist,muslim_min_dist,christian_min_dist,buddhist_min_dist,dist_to_cc,foreign_nationals,unemployment_rate,income_levels,random_noise,geometry
0,marseille,FR-official_id-130020106-admin_level-11,-0.444444,-0.187919,-0.300000,0.0,-0.571429,-0.810811,-0.444444,-0.622222,...,2.841163,2.829058,0.903775,-0.176350,1.454689,-0.684105,-0.521390,1.029202,1.334103,"POLYGON ((5.54771 43.34452, 5.54753 43.34451, ..."
1,marseille,FR-official_id-130050702-admin_level-11,-0.333333,-0.161074,0.200000,0.0,-0.642857,-0.756757,-0.444444,-0.533333,...,4.198778,4.817203,1.997066,0.033113,2.122303,-0.669459,-0.619313,1.074703,1.612249,"POLYGON ((5.51860 43.28693, 5.51839 43.28692, ..."
2,marseille,FR-official_id-130710105-admin_level-11,-0.333333,-0.080537,-0.200000,0.0,-0.285714,-0.864865,-0.111111,-0.355556,...,3.466202,0.986584,1.253323,-1.716229,1.267491,-0.703629,-0.573965,0.504457,3.058110,"POLYGON ((5.35367 43.37689, 5.35352 43.37659, ..."
3,marseille,FR-official_id-130750102-admin_level-11,-0.444444,-0.107383,-0.400000,0.0,-0.428571,-0.108108,-0.333333,-0.444444,...,1.389558,1.039531,0.382529,0.341271,0.925596,-0.676339,-0.598862,0.612597,0.505580,"POLYGON ((5.47010 43.34248, 5.47001 43.34236, ..."
4,marseille,FR-official_id-130750104-admin_level-11,-0.444444,-0.187919,-0.400000,0.0,-0.571429,-0.324324,-0.444444,-0.444444,...,2.477888,2.057345,2.331364,0.880692,1.379049,-0.691519,-0.454218,0.928744,1.677045,"POLYGON ((5.46373 43.34745, 5.46349 43.34758, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752,bremen,DE-official_id-04012000021-admin_level-11,-0.333333,0.000000,-0.470588,0.0,-0.500000,-0.571429,-0.500000,-0.166667,...,0.231711,0.428310,1.972046,1.890189,1.928484,-0.230593,0.339796,0.320310,1.967400,"POLYGON ((8.63110 53.55557, 8.63111 53.55555, ..."
1753,bremen,DE-official_id-04012000022-admin_level-11,0.000000,3.000000,-0.352941,3.0,-0.500000,-0.428571,0.000000,-0.500000,...,-0.741527,0.497816,0.866531,2.180809,2.220471,-0.648528,0.037946,0.463917,1.187478,"POLYGON ((8.59095 53.59348, 8.59097 53.59336, ..."
1754,bremen,DE-official_id-04012000023-admin_level-11,-0.333333,0.000000,-0.470588,0.0,-1.000000,-0.714286,-0.500000,-0.500000,...,0.570909,0.510213,-0.230222,1.836003,1.873291,-0.565739,0.610655,-0.051318,1.753039,"POLYGON ((8.63762 53.50365, 8.63758 53.50363, ..."
1755,bremen,DE-official_id-04012000024-admin_level-11,0.333333,0.000000,0.000000,0.0,1.000000,0.000000,0.500000,0.333333,...,-0.720531,-0.654242,-0.060436,2.079863,2.117092,0.881500,1.951820,-1.320075,-0.106370,"POLYGON ((8.58696 53.56894, 8.58672 53.56694, ..."


### Leave-one-city-out

In [8]:
loo_results_impr = pd.DataFrame(columns = ['unemployment_rate', 'foreign_nationals', 'income_levels'])
loo_results_r2_score = pd.DataFrame(columns = ['unemployment_rate', 'foreign_nationals', 'income_levels'])
p_values = pd.DataFrame(columns = ['unemployment_rate_p_val', 'foreign_nationals_p_val', 'income_levels_p_val'])

#### Results

In [9]:
for city in ['marseille', 'lyon', 'paris']: 
    
    country = 'FR'
    target = 'unemployment_rate'
    target_city = city
    
    train = agg_full[agg_full.assigned_city!=target_city]
    test = agg_full[agg_full.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights
    

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    loo_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    loo_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    

    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.unemployment_rate-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.unemployment_rate-predicts.mean_model)**2
    print(residuals.shape)
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_values.loc[city,f'{target}_p_val']=p
    
for city in ['berlin', 'hamburg', 'bremen']:
    country = 'DE'
    target_city = city
    
    train = agg_full[agg_full.assigned_city!=target_city]
    test = agg_full[agg_full.assigned_city==target_city]

    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights    
    
    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    loo_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    loo_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)

     
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.unemployment_rate-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.unemployment_rate-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (1413, 236)
number of pca components: 173
shape after pca: (1413, 173)
0.1
shape before pca: (1413, 236)
number of pca components: 173
shape after pca: (1413, 173)
0.1
(344, 2)
TTest for residuals of model in marseille: t=1.898853321132751, p=0.058003083347193306
shape before pca: (1581, 236)
number of pca components: 169
shape after pca: (1581, 169)
0.2
shape before pca: (1581, 236)
number of pca components: 169
shape after pca: (1581, 169)
0.2
(176, 2)
TTest for residuals of model in lyon: t=0.0017878635870715897, p=0.9985745105782624
shape before pca: (896, 236)
number of pca components: 146
shape after pca: (896, 146)
0.01
shape before pca: (896, 236)
number of pca components: 146
shape after pca: (896, 146)
0.01
(861, 2)
TTest for residuals of model in paris: t=1.4353507698166532, p=0.15136898334939428
shape before pca: (1635, 236)
number of pca components: 167
shape after pca: (1635, 167)
0.1
shape before pca: (1635, 236)
number of pca components: 167
shape afte

In [10]:
for city in ['marseille', 'lyon', 'paris']: 
    
    country = 'FR'
    target = 'income_levels'
    target_city = city
    
    train = agg_full[agg_full.assigned_city!=target_city]
    test = agg_full[agg_full.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights
    

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    loo_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    loo_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.income_levels-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.income_levels-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}') 
    p_values.loc[city,f'{target}_p_val']=p
    
for city in ['berlin', 'hamburg', 'bremen']:
    country = 'DE'
    target_city = city
    
    train = agg_full[agg_full.assigned_city!=target_city]
    test = agg_full[agg_full.assigned_city==target_city]

    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights    
    
    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    loo_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    loo_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.income_levels-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.income_levels-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')  
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (1413, 236)
number of pca components: 173
shape after pca: (1413, 173)
0.01
shape before pca: (1413, 236)
number of pca components: 173
shape after pca: (1413, 173)
0.01
TTest for residuals of model in marseille: t=2.0345441813893816, p=0.042280848870142364
shape before pca: (1581, 236)
number of pca components: 169
shape after pca: (1581, 169)
0.1
shape before pca: (1581, 236)
number of pca components: 169
shape after pca: (1581, 169)
0.1
TTest for residuals of model in lyon: t=-0.3461174750860629, p=0.7294623959217803
shape before pca: (896, 236)
number of pca components: 146
shape after pca: (896, 146)
0.01
shape before pca: (896, 236)
number of pca components: 146
shape after pca: (896, 146)
0.01
TTest for residuals of model in paris: t=1.488209765644964, p=0.13687889124352362
shape before pca: (1635, 236)
number of pca components: 167
shape after pca: (1635, 167)
0.1
shape before pca: (1635, 236)
number of pca components: 167
shape after pca: (1635, 167)
0.1
TTes

In [11]:
for city in ['marseille', 'lyon', 'paris']: 
    
    country = 'FR'
    target = 'foreign_nationals'
    target_city = city
    
    train = agg_full[agg_full.assigned_city!=target_city]
    test = agg_full[agg_full.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights
    

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True)
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    loo_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    loo_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.foreign_nationals-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.foreign_nationals-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')   
    p_values.loc[city,f'{target}_p_val']=p
    
for city in ['berlin', 'hamburg', 'bremen']:
    country = 'DE'
    target_city = city
    
    train = agg_full[agg_full.assigned_city!=target_city]
    test = agg_full[agg_full.assigned_city==target_city]

    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights    
    
    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True)
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    loo_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    loo_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.foreign_nationals-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.foreign_nationals-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}') 
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (1413, 236)
number of pca components: 173
shape after pca: (1413, 173)
0.01
shape before pca: (1413, 236)
number of pca components: 173
shape after pca: (1413, 173)
0.01
TTest for residuals of model in marseille: t=2.0027927357560644, p=0.0455925635706273
shape before pca: (1581, 236)
number of pca components: 169
shape after pca: (1581, 169)
0.2
shape before pca: (1581, 236)
number of pca components: 169
shape after pca: (1581, 169)
0.2
TTest for residuals of model in lyon: t=-0.05536683977432134, p=0.9558778047617917
shape before pca: (896, 236)
number of pca components: 146
shape after pca: (896, 146)
0.01
shape before pca: (896, 236)
number of pca components: 146
shape after pca: (896, 146)
0.01
TTest for residuals of model in paris: t=1.59359240480761, p=0.11121108970023609
shape before pca: (1635, 236)
number of pca components: 167
shape after pca: (1635, 167)
0.1
shape before pca: (1635, 236)
number of pca components: 167
shape after pca: (1635, 167)
0.1
TTest 

In [7]:
loo_results_impr.loc['mean'] = loo_results_impr.mean()
loo_results_impr

,unemployment_rate,foreign_nationals,income_levels
marseille,25.676754,31.633758,19.183957
lyon,0.048349,-1.309613,-6.150607
paris,13.251059,13.005593,9.975073
berlin,40.128826,33.333416,20.37572
hamburg,39.076534,19.465109,20.52393
bremen,25.784639,5.899169,15.773473
mean,23.99436,17.004572,13.280258


In [8]:
loo_results_r2_score.loc['mean'] = loo_results_r2_score.mean()
loo_results_r2_score

,unemployment_rate,foreign_nationals,income_levels
marseille,0.245659,0.302784,0.191492
lyon,-0.004347,-0.013445,-0.063776
paris,0.127158,0.119054,0.099139
berlin,0.336146,0.320267,0.201038
hamburg,0.325853,0.19186,0.20353
bremen,0.228731,0.009735,0.131206
mean,0.209867,0.155042,0.127105


In [12]:
p_values

,unemployment_rate_p_val,foreign_nationals_p_val,income_levels_p_val
marseille,0.058003,0.045593,0.042281
lyon,0.998575,0.955878,0.729462
paris,0.151369,0.111211,0.136879
berlin,0.003382,0.19615,0.120556
hamburg,0.02833,0.584531,0.142389
bremen,0.123868,0.70064,0.444464


### Leave-one-city-in-country-out

In [13]:
locico_results_impr = pd.DataFrame(columns = ['unemployment_rate', 'foreign_nationals', 'income_levels'])
locico_results_r2_score = pd.DataFrame(columns = ['unemployment_rate', 'foreign_nationals', 'income_levels'])
p_values = pd.DataFrame(columns = ['unemployment_rate_p_val', 'foreign_nationals_p_val', 'income_levels_p_val'])

### France

In [14]:
print(agg_full.shape)
agg_full_fr = agg_full[agg_full.assigned_city.isin(['marseille', 'lyon', 'paris'])]
print(agg_full_fr.shape)

(1757, 243)
(1381, 243)


In [15]:
for city in ['marseille', 'lyon', 'paris']: 
    
    country = 'FR'
    target = 'unemployment_rate'
    target_city = city
    
    train = agg_full_fr[agg_full_fr.assigned_city!=target_city]
    test = agg_full_fr[agg_full_fr.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    if (len(cols_lasso)==0):
        boosted_lasso_predicts = clf1.predict(X_test)
    else:
        boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
        boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    locico_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    locico_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.unemployment_rate-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.unemployment_rate-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (1037, 236)
number of pca components: 146
shape after pca: (1037, 146)
0.1
shape before pca: (1037, 236)
number of pca components: 146
shape after pca: (1037, 146)
0.1
TTest for residuals of model in marseille: t=1.3230136663038228, p=0.18627165128227308
shape before pca: (1205, 236)
number of pca components: 144
shape after pca: (1205, 144)
0.2
shape before pca: (1205, 236)
number of pca components: 144
shape after pca: (1205, 144)
0.2
TTest for residuals of model in lyon: t=0.3037212368564033, p=0.7615207491241005
shape before pca: (520, 236)
number of pca components: 98
shape after pca: (520, 98)
0.01
shape before pca: (520, 236)
number of pca components: 98
shape after pca: (520, 98)
0.01
TTest for residuals of model in paris: t=1.2693300536771595, p=0.2044950959192632


In [21]:
for city in ['marseille', 'lyon', 'paris']: 
    
    country = 'FR'
    target = 'income_levels'
    target_city = city
    
    train = agg_full_fr[agg_full_fr.assigned_city!=target_city]
    test = agg_full_fr[agg_full_fr.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    if (len(cols_lasso)==0):
        boosted_lasso_predicts = clf1.predict(X_test)
    else:
        boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
        boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    locico_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    locico_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.income_levels-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.income_levels-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')     
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (1037, 236)
number of pca components: 146
shape after pca: (1037, 146)
0.01
shape before pca: (1037, 236)
number of pca components: 146
shape after pca: (1037, 146)
0.01
TTest for residuals of model in marseille: t=2.9052535921166522, p=0.0037876991160248497
shape before pca: (1205, 236)
number of pca components: 144
shape after pca: (1205, 144)
0.1
shape before pca: (1205, 236)
number of pca components: 144
shape after pca: (1205, 144)
0.1
TTest for residuals of model in lyon: t=0.24498388583512465, p=0.8066124796234537
shape before pca: (520, 236)
number of pca components: 98
shape after pca: (520, 98)
0.01
shape before pca: (520, 236)
number of pca components: 98
shape after pca: (520, 98)
0.01


XGBoostError: [11:19:39] ..\src\learner.cc:599: Check failed: mparam_.num_feature != 0 (0 vs. 0) : 0 feature is supplied.  Are you using raw Booster interface?

In [16]:
for city in ['marseille', 'lyon', 'paris']: 
    
    country = 'FR'
    target = 'foreign_nationals'
    target_city = city
    
    train = agg_full_fr[agg_full_fr.assigned_city!=target_city]
    test = agg_full_fr[agg_full_fr.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    locico_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    locico_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.foreign_nationals-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.foreign_nationals-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}') 
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (1037, 236)
number of pca components: 146
shape after pca: (1037, 146)
0.01
shape before pca: (1037, 236)
number of pca components: 146
shape after pca: (1037, 146)
0.01
TTest for residuals of model in marseille: t=-0.41681624771306214, p=0.676943209887388
shape before pca: (1205, 236)
number of pca components: 144
shape after pca: (1205, 144)
0.2
shape before pca: (1205, 236)
number of pca components: 144
shape after pca: (1205, 144)
0.2
TTest for residuals of model in lyon: t=-0.06250258523033946, p=0.9501982811204028
shape before pca: (520, 236)
number of pca components: 98
shape after pca: (520, 98)
0.01
shape before pca: (520, 236)
number of pca components: 98
shape after pca: (520, 98)
0.01
TTest for residuals of model in paris: t=-0.21151064864913552, p=0.832513881590716


### germany

In [17]:
print(agg_full.shape)
agg_full_de = agg_full[agg_full.assigned_city.isin(['berlin', 'hamburg', 'bremen'])]
print(agg_full_de.shape)

(1757, 243)
(376, 243)


In [18]:
target = 'unemployment_rate'
for city in ['berlin', 'hamburg', 'bremen']: 
    
    country = 'DE'
    target_city = city
    
    train = agg_full_de[agg_full_de.assigned_city!=target_city]
    test = agg_full_de[agg_full_de.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True)
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    locico_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    locico_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.unemployment_rate-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.unemployment_rate-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (254, 236)
number of pca components: 91
shape after pca: (254, 91)
0.1
shape before pca: (254, 236)
number of pca components: 91
shape after pca: (254, 91)
0.1
TTest for residuals of model in berlin: t=2.2026775724749355, p=0.028560781810677647
shape before pca: (251, 236)
number of pca components: 106
shape after pca: (251, 106)
0.1
shape before pca: (251, 236)
number of pca components: 106
shape after pca: (251, 106)
0.1
TTest for residuals of model in hamburg: t=1.692549138716023, p=0.09179716897152644
shape before pca: (247, 236)
number of pca components: 95
shape after pca: (247, 95)
0.1
shape before pca: (247, 236)
number of pca components: 95
shape after pca: (247, 95)
0.1
TTest for residuals of model in bremen: t=0.7263334794355241, p=0.46829792708386975


In [19]:
target = 'income_levels'
for city in ['berlin', 'hamburg', 'bremen']: 
    
    country = 'DE'
    target_city = city
    
    train = agg_full_de[agg_full_de.assigned_city!=target_city]
    test = agg_full_de[agg_full_de.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    locico_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    locico_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.income_levels-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.income_levels-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')  
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (254, 236)
number of pca components: 91
shape after pca: (254, 91)
0.1
shape before pca: (254, 236)
number of pca components: 91
shape after pca: (254, 91)
0.1
TTest for residuals of model in berlin: t=1.9787345747447063, p=0.04897972023257096
shape before pca: (251, 236)
number of pca components: 106
shape after pca: (251, 106)
0.1
shape before pca: (251, 236)
number of pca components: 106
shape after pca: (251, 106)
0.1
TTest for residuals of model in hamburg: t=1.1159700241817525, p=0.2655150932154268
shape before pca: (247, 236)
number of pca components: 95
shape after pca: (247, 95)
0.2
shape before pca: (247, 236)
number of pca components: 95
shape after pca: (247, 95)
0.2
TTest for residuals of model in bremen: t=0.6661827924149295, p=0.5058941436683685


In [20]:
target = 'foreign_nationals'
for city in ['berlin', 'hamburg', 'bremen']: 
    
    country = 'DE'
    target_city = city
    
    train = agg_full_de[agg_full_de.assigned_city!=target_city]
    test = agg_full_de[agg_full_de.assigned_city==target_city]
    
    weights_train = []
    for train_city in train.assigned_city.unique().tolist():
        subset = train[train.assigned_city == train_city]
        weights = [1 - (len(subset)/len(train))]*len(subset)
        weights_train = weights_train + weights

    X_train = train.iloc[:,2:-5]
    y_train = train.loc[:,[target]]
    X_test = test.iloc[:,2:-5]
    y_test = test.loc[:,[target]].reset_index(drop = True) 
    
    # Boosted Lasso predicts
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf1 = GridSearchCV(lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf1.fit(X_train, y_train, sample_weight= weights_train)
    
    cols_lasso = X_train.loc[:,clf1.best_estimator_.coef_!=0].columns.tolist()
    boosted_lasso = train_xgboost(train, target, cols_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    boosted_lasso_predicts = boosted_lasso.predict(X_test[cols_lasso])
    
    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=train,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=False)
    pca = get_best_pca_lasso_model(agg=train ,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=False)
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(train.iloc[:,-5:].reset_index(drop = True))
    
    alphas = [0.01, 0.1,0.2,0.3,0.4,0.5, 0.6,0.7,0.8,0.9,1,2,3,5,6,7,8,9,10]
    pca_lasso = linear_model.Lasso(max_iter = 50000)
    parameters = {'alpha':alphas}
    clf2 = GridSearchCV(pca_lasso, parameters, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
    clf2.fit(reduced_data.iloc[:,:-5], reduced_data.loc[:,[target]], sample_weight= weights_train)
    cols_pca_lasso = reduced_data.iloc[:,:-5].loc[:,clf2.best_estimator_.coef_!=0].columns.tolist()
    boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier',learner_type = 'transfer', weights = weights_train)
    
    reduced_test_data = pd.DataFrame(pca.transform(X_test))
    boosted_pca_lasso_predicts = boosted_pca_lasso.predict(reduced_test_data[cols_pca_lasso])
    
    predicts = pd.DataFrame({'boosted_lasso': boosted_lasso_predicts, 'boosted_pca_lasso': boosted_pca_lasso_predicts})
    predicts.loc[:,'mean_model'] = predicts.mean(axis = 1)

    naive_pred = [y_train.mean().values[0]] * len(y_test)
    locico_results_impr.loc[city, target] = 100-(metrics.mean_squared_error(y_test, predicts.mean_model)/metrics.mean_squared_error(y_test, naive_pred)*100)
    locico_results_r2_score.loc[city, target] = metrics.r2_score(y_test, predicts.mean_model)
    
    residuals = pd.DataFrame(columns = ['residuals_naive', 'residuals_m5'])
    residuals.loc[:,'residuals_naive'] = (y_test.foreign_nationals-naive_pred)**2
    residuals.loc[:,'residuals_m5'] = (y_test.foreign_nationals-predicts.mean_model)**2
    
    t_stat, p = stats.ttest_ind(residuals['residuals_naive'], residuals['residuals_m5'])
    print(f'TTest for residuals of model in {city}: t={t_stat}, p={p}')    
    p_values.loc[city,f'{target}_p_val']=p

shape before pca: (254, 236)
number of pca components: 91
shape after pca: (254, 91)
0.1
shape before pca: (254, 236)
number of pca components: 91
shape after pca: (254, 91)
0.1
TTest for residuals of model in berlin: t=0.9500339068627435, p=0.3430428325313237
shape before pca: (251, 236)
number of pca components: 106
shape after pca: (251, 106)
0.5
shape before pca: (251, 236)
number of pca components: 106
shape after pca: (251, 106)
0.5
TTest for residuals of model in hamburg: t=0.5350606634667227, p=0.5930873364921916
shape before pca: (247, 236)
number of pca components: 95
shape after pca: (247, 95)
0.5
shape before pca: (247, 236)
number of pca components: 95
shape after pca: (247, 95)
0.5
TTest for residuals of model in bremen: t=-0.9556310630141894, p=0.3401601934424936


In [18]:
locico_results_impr.loc['mean'] = locico_results_impr.mean()
locico_results_impr

,unemployment_rate,foreign_nationals,income_levels
marseille,17.468847,-7.680231,26.136523
lyon,7.783741,-1.566205,4.194064
paris,11.678524,-1.900219,NaN
berlin,32.501895,26.161711,26.487949
hamburg,33.993034,18.685804,15.942349
bremen,14.290642,-18.533477,13.959362
mean,19.619447,2.527897,17.344049


In [19]:
locico_results_r2_score.loc['mean'] = locico_results_r2_score.mean()
locico_results_r2_score

,unemployment_rate,foreign_nationals,income_levels
marseille,0.172597,-0.102659,0.261354
lyon,0.077668,-0.015891,0.041089
paris,0.114996,-0.03962,NaN
berlin,0.324856,0.239313,0.233561
hamburg,0.338173,0.183433,0.157895
bremen,0.140561,-0.331548,0.116151
mean,0.194808,-0.011162,0.16201


In [22]:
p_values

,unemployment_rate_p_val,foreign_nationals_p_val,income_levels_p_val
marseille,0.186272,0.676943,0.003788
lyon,0.761521,0.950198,0.806612
paris,0.204495,0.832514,NaN
berlin,0.028561,0.343043,0.04898
hamburg,0.091797,0.593087,0.265515
bremen,0.468298,0.34016,0.505894


### All to all city prediction

In [ ]:
improvement_matrix = pd.DataFrame(columns = ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen'])
r2_score_matrix = pd.DataFrame(columns = ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen'])

target = 'unemployment_rate'
for train_city in ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen']:
    predicts_master = pd.DataFrame(columns = ['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted', 'y_pred_mean_model', 'y_pred_naive'])
    if (train_city in ['marseille', 'lyon', 'paris']):
        country = 'FR'
    else:
        country = 'DE'
    agg_subset = agg_full[agg_full.assigned_city==train_city].reset_index(drop = True)

    
    # Lasso Boosted
    cols_lasso = get_best_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015, density_type='count', radius = 1000, output ='used_columns')
    predicts = train_xgboost(agg_subset, target, cols_lasso, 'predicts')
    lasso_xgb = train_xgboost(agg_subset, target, cols_lasso, 'classifier')
    
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred
    predicts_master.loc[:, 'y_pred_naive'] = predicts.naive

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components')
    pca = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier')
    cols_pca_lasso = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg_subset.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols_pca_lasso, 'predicts')
    pca_lasso_xgb = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_test'] = predicts.y_test
    
    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_naive) 
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    
    improvement_matrix.loc[train_city, train_city] = improvement_mean_model
    r2_score_matrix.loc[train_city, train_city] = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    for target_city in ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen']:
        
        if(target_city != train_city):
            
             
            if (target_city in ['marseille', 'lyon', 'paris']):
                country = 'FR'
            else:
                country = 'DE'
            
            predicts_master = pd.DataFrame(columns = ['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted', 'y_pred_mean_model', 'y_pred_naive'])
            test_data = agg_full[agg_full.assigned_city==target_city].reset_index(drop = True)

            predicts_master.loc[:,'y_pred_lasso_boosted'] = lasso_xgb.predict(test_data.iloc[:,2:-5][cols_lasso])

            reduced_data = pd.DataFrame(pca.transform(test_data.iloc[:,2:-5]))
            reduced_data = reduced_data.join(test_data.iloc[:,-5:])
            predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = pca_lasso_xgb.predict(reduced_data.iloc[:,:-5][cols_pca_lasso])

            predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
            predicts_master.loc[:, 'y_test'] = test_data[target]
            predicts_master.loc[:, 'y_pred_naive'] = test_data[target].mean()


            naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_naive) 
            mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
            improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100

            improvement_matrix.loc[train_city, target_city] = improvement_mean_model
            r2_score_matrix.loc[train_city, target_city] = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

In [ ]:
improvement_matrix

In [ ]:
r2_score_matrix

In [ ]:
# Generate a custom diverging colormap
#cmap = sns.diverging_palette(10, 133, as_cmap=True)

sns.heatmap(improvement_matrix.astype(float), cmap='viridis', vmin=improvement_matrix.min().min(), vmax=improvement_matrix.max().max(),
            center=0.00, square=True, linewidths=.5, annot_kws={"size": 8}, annot=True)

In [ ]:
# Generate a custom diverging colormap
#cmap = sns.diverging_palette(10, 133, as_cmap=True)

sns.heatmap(r2_score_matrix.astype(float), cmap='viridis', vmin=r2_score_matrix.min().min(), vmax=r2_score_matrix.max().max(),
            center=0.00, square=True, linewidths=.5, annot_kws={"size": 8}, annot=True)

In [ ]:
improvement_matrix = pd.DataFrame(columns = ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen'])
r2_score_matrix = pd.DataFrame(columns = ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen'])

target = 'foreign_nationals'
for train_city in ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen']:
    predicts_master = pd.DataFrame(columns = ['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted', 'y_pred_mean_model', 'y_pred_naive'])
    if (train_city in ['marseille', 'lyon', 'paris']):
        country = 'FR'
    else:
        country = 'DE'
    agg_subset = agg_full[agg_full.assigned_city==train_city].reset_index(drop = True)

    
    # Lasso Boosted
    cols_lasso = get_best_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015, density_type='count', radius = 1000, output ='used_columns')
    predicts = train_xgboost(agg_subset, target, cols_lasso, 'predicts')
    lasso_xgb = train_xgboost(agg_subset, target, cols_lasso, 'classifier')
    
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred
    predicts_master.loc[:, 'y_pred_naive'] = predicts.naive

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components')
    pca = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier')
    cols_pca_lasso = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg_subset.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols_pca_lasso, 'predicts')
    pca_lasso_xgb = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_test'] = predicts.y_test
    
    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_naive) 
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    
    improvement_matrix.loc[train_city, train_city] = improvement_mean_model
    r2_score_matrix.loc[train_city, train_city] = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    for target_city in ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen']:
        
        if(target_city != train_city):
            
             
            if (target_city in ['marseille', 'lyon', 'paris']):
                country = 'FR'
            else:
                country = 'DE'
            
            predicts_master = pd.DataFrame(columns = ['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted', 'y_pred_mean_model', 'y_pred_naive'])
            test_data = agg_full[agg_full.assigned_city==target_city].reset_index(drop = True)

            predicts_master.loc[:,'y_pred_lasso_boosted'] = lasso_xgb.predict(test_data.iloc[:,2:-5][cols_lasso])

            reduced_data = pd.DataFrame(pca.transform(test_data.iloc[:,2:-5]))
            reduced_data = reduced_data.join(test_data.iloc[:,-5:])
            predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = pca_lasso_xgb.predict(reduced_data.iloc[:,:-5][cols_pca_lasso])

            predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
            predicts_master.loc[:, 'y_test'] = test_data[target]
            predicts_master.loc[:, 'y_pred_naive'] = test_data[target].mean()


            naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_naive) 
            mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
            improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100

            improvement_matrix.loc[train_city, target_city] = improvement_mean_model
            r2_score_matrix.loc[train_city, target_city] = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

In [ ]:
# Generate a custom diverging colormap
#cmap = sns.diverging_palette(10, 133, as_cmap=True)

sns.heatmap(improvement_matrix.astype(float), cmap='viridis', vmin=improvement_matrix.min().min(), vmax=improvement_matrix.max().max(),
            center=0.00, square=True, linewidths=.5, annot_kws={"size": 8}, annot=True)

In [ ]:
sns.heatmap(r2_score_matrix.astype(float), cmap='viridis', vmin=r2_score_matrix.min().min(), vmax=r2_score_matrix.max().max(),
            center=0.00, square=True, linewidths=.5, annot_kws={"size": 8}, annot=True)

In [ ]:
improvement_matrix = pd.DataFrame(columns = ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen'])
r2_score_matrix = pd.DataFrame(columns = ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen'])

target = 'income_levels'
for train_city in ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen']:
    predicts_master = pd.DataFrame(columns = ['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted', 'y_pred_mean_model', 'y_pred_naive'])
    if (train_city in ['marseille', 'lyon', 'paris']):
        country = 'FR'
    else:
        country = 'DE'
    agg_subset = agg_full[agg_full.assigned_city==train_city].reset_index(drop = True)

    
    # Lasso Boosted
    cols_lasso = get_best_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns')
    predicts = train_xgboost(agg_subset, target, cols_lasso, 'predicts')
    lasso_xgb = train_xgboost(agg_subset, target, cols_lasso, 'classifier')
    
    predicts_master.loc[:, 'y_pred_lasso_boosted'] = predicts.y_pred
    predicts_master.loc[:, 'y_pred_naive'] = predicts.naive

    # PCA Lasso boosted
    comps = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components')
    pca = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier')
    cols_pca_lasso = get_best_pca_lasso_model(agg=agg_subset, target=target, city=train_city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns')
    reduced_data = pd.DataFrame(comps)
    reduced_data = reduced_data.join(agg_subset.iloc[:,-5:])
    predicts = train_xgboost(reduced_data, target, cols_pca_lasso, 'predicts')
    pca_lasso_xgb = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier')
    predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = predicts.y_pred

    predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
    predicts_master.loc[:, 'y_test'] = predicts.y_test
    
    naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_naive) 
    mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
    improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100
    
    improvement_matrix.loc[train_city, train_city] = improvement_mean_model
    r2_score_matrix.loc[train_city, train_city] = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

    for target_city in ['lyon', 'paris', 'marseille', 'berlin', 'hamburg', 'bremen']:
        
        if(target_city != train_city):
            
             
            if (target_city in ['marseille', 'lyon', 'paris']):
                country = 'FR'
            else:
                country = 'DE'
            
            predicts_master = pd.DataFrame(columns = ['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted', 'y_pred_mean_model', 'y_pred_naive'])
            test_data = agg_full[agg_full.assigned_city==target_city].reset_index(drop = True)

            predicts_master.loc[:,'y_pred_lasso_boosted'] = lasso_xgb.predict(test_data.iloc[:,2:-5][cols_lasso])

            reduced_data = pd.DataFrame(pca.transform(test_data.iloc[:,2:-5]))
            reduced_data = reduced_data.join(test_data.iloc[:,-5:])
            predicts_master.loc[:,'y_pred_pca_lasso_boosted'] = pca_lasso_xgb.predict(reduced_data.iloc[:,:-5][cols_pca_lasso])

            predicts_master.loc[:, 'y_pred_mean_model'] = predicts_master[['y_pred_lasso_boosted', 'y_pred_pca_lasso_boosted']].mean(axis = 1)
            predicts_master.loc[:, 'y_test'] = test_data[target]
            predicts_master.loc[:, 'y_pred_naive'] = test_data[target].mean()


            naive_mse = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_naive) 
            mse_mean_model = metrics.mean_squared_error(predicts_master.y_test, predicts_master.y_pred_mean_model)
            improvement_mean_model = 100 - (mse_mean_model/naive_mse)*100

            improvement_matrix.loc[train_city, target_city] = improvement_mean_model
            r2_score_matrix.loc[train_city, target_city] = metrics.r2_score(predicts_master.y_test, predicts_master.y_pred_mean_model)

In [ ]:
# Generate a custom diverging colormap
#cmap = sns.diverging_palette(10, 133, as_cmap=True)

sns.heatmap(improvement_matrix.astype(float), cmap='viridis', vmin=improvement_matrix.min().min(), vmax=improvement_matrix.max().max(),
            center=0.00, square=True, linewidths=.5, annot_kws={"size": 8}, annot=True)

In [ ]:
sns.heatmap(r2_score_matrix.astype(float), cmap='viridis', vmin=r2_score_matrix.min().min(), vmax=r2_score_matrix.max().max(),
            center=0.00, square=True, linewidths=.5, annot_kws={"size": 8}, annot=True)

## Meta Learner trained on all cities

In [20]:
cities_fr = ['marseille', 'lyon', 'paris']
cities_de = ['berlin', 'hamburg', 'bremen']

agg_full = pd.DataFrame()

if (len(cities_fr) !=0):
    for city in cities_fr:
        agg = get_training_data(city, 'FR', 1000, 'count', 2015)
        agg_full = agg_full.append(agg)
        
if (len(cities_de) !=0):
    for city in cities_de:
        agg = get_training_data(city, 'DE', 1000, 'count', 2015)
        agg_full = agg_full.append(agg)

    
print(agg_full.shape)
agg_full=agg_full.dropna(axis = 1)
agg_full = agg_full.reset_index(drop = True)
print(agg_full.shape)

shape of training data (344, 243)
shape of training data (176, 243)
shape of training data (861, 243)
shape of training data (122, 243)
shape of training data (125, 243)
shape of training data (129, 243)
(1757, 243)
(1757, 243)


In [21]:
meta_learner_results = pd.DataFrame(columns = ['unemployment_rate', 'foreign_nationals', 'income_levels'])
meta_learner_r2_score = pd.DataFrame(columns = ['unemployment_rate', 'foreign_nationals', 'income_levels'])

In [22]:
target = 'unemployment_rate'

for target_city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:

    train_full = pd.DataFrame(columns = agg_full.columns)
    meta_train_full = pd.DataFrame(columns = agg_full.columns)
    test_full = pd.DataFrame(columns = agg_full.columns)

    # filter data for respective city
    for city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:


        subset = agg_full[agg_full.assigned_city == city]

        if (target_city != city):
        # create three subsets - training data, meta_training_data, test_data
            train, meta_train = train_test_split(subset, test_size=30, random_state=41)
            # append created datasets
            train_full = train_full.append(train)
            meta_train_full = meta_train_full.append(meta_train)

        else:
            test_full = subset

    #reset index for all three subsets
    train_full = train_full.reset_index(drop = True)
    meta_train_full = meta_train_full.reset_index(drop = True)
    test_full = test_full.reset_index(drop = True)

    print('shape of training data: '+str(train_full.shape))
    print('shape of meta training data: '+str(meta_train_full.shape))
    print('shape of test data: '+str(test_full.shape))

    cities = ['marseille', 'lyon', 'paris', 'hamburg', 'bremen', 'berlin', target]
    cities.remove(target_city)

    meta_learner_train = pd.DataFrame(columns = cities)
    meta_learner_test = pd.DataFrame(columns = cities)

    # split data in feature and target
    X_meta_train_full = meta_train_full.iloc[:,2:-5]
    y_meta_train_full = meta_train_full[[target]]

    X_test_full = test_full.iloc[:,2:-5]
    y_test_full = test_full[[target]]

    for city in ['marseille', 'lyon', 'paris']:
        country = 'FR'

        if(city != target_city):
            #filter out data of respective city
            city_training_data = train_full[train_full.assigned_city == city]

            # Lasso Boosted
            #get columns lasso model selects
            cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
            # get xgboost classifier
            boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier', learner_type = 'meta')
            #make prediction for meta_train data and test data
            boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
            boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

            # PCA Lasso boosted
            # get the components pca returns
            comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=True)
            # get columns to selct the most important components from pca
            cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
            # get fitted pca model
            pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
            #create reduced training data based on pca components
            reduced_data = pd.DataFrame(comps)
            reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
            #get trained xgboost
            boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'meta')

            #apply pca to meta_train data
            print('meta full: '+str(meta_train_full.shape))
            reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
            print('reduced pca: '+str(reduced_X_meta_train_full.shape))

            # apply pca to test data
            print('test full: '+str(test_full.shape))
            reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
            print('reduced pca: '+str(reduced_X_test_full.shape))

            # make predictions for meta_train data and test data
            boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
            boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

            #store predictions for meta_train in new dataframe and get the mean value of both predictions
            city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
            city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
            city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            #store predictions for test_data in new dataframe and get the mean value of both predictions
            city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
            city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
            city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            # add prediction to full data sets for meta_training and test
            meta_learner_train.loc[:,city] = city_result_meta['mean_model']
            meta_learner_test.loc[:,city] = city_result_test['mean_model']

    for city in ['bremen', 'hamburg', 'berlin']:

        if (city != target_city):

            country = 'DE'

            city_training_data = train_full[train_full.assigned_city == city]

            # Lasso Boosted
            cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
            boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier', learner_type = 'meta')
            boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
            boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

            # PCA Lasso boosted
            comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
            cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
            pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
            reduced_data = pd.DataFrame(comps)
            reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
            boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'meta')

            print('meta full: '+str(meta_train_full.shape))
            reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
            print('reduced pca: '+str(reduced_X_meta_train_full.shape))

            print('test full: '+str(test_full.shape))
            reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
            print('reduced pca: '+str(reduced_X_test_full.shape))

            boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
            boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

            city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
            city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
            city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
            city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
            city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            meta_learner_train.loc[:,city] = city_result_meta['mean_model']
            meta_learner_test.loc[:,city] = city_result_test['mean_model']   

    # add the target column to the meta_train set and test set and add cities back to dataframe    
    meta_learner_train.loc[:,target]= y_meta_train_full
    meta_learner_test.loc[:,target] = y_test_full
    meta_learner_train.loc[:, 'assigned_city'] = meta_train_full.assigned_city
    meta_learner_test.loc[:, 'assigned_city'] = test_full.assigned_city


    clf = LinearRegression()
    clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

    #make prediction for test data
    y_pred = clf.predict(meta_learner_test.iloc[:,:-2])
    naive_pred = [meta_learner_train[[target]].mean().values[0]] * len(meta_learner_test[[target]])

    meta_learner_results.loc[target_city, target] = (100-(metrics.mean_squared_error(meta_learner_test[[target]], y_pred)/metrics.mean_squared_error(meta_learner_test[[target]], naive_pred)*100))
    meta_learner_r2_score.loc[target_city, target] = metrics.r2_score(meta_learner_test[[target]], y_pred)

shape of training data: (1263, 243)
shape of meta training data: (150, 243)
shape of test data: (344, 243)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (344, 243)
reduced pca: (344, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (344, 243)
reduced pca: (344, 133)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (125, 243)
reduced pca: (125, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (125, 243)
reduced pca: (125, 133)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
meta full: (150, 243)
reduced pca: (150, 53)
test full: (125, 243)
reduced pca: (125, 53)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
sh

In [23]:
target = 'foreign_nationals'

for target_city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:

    train_full = pd.DataFrame(columns = agg_full.columns)
    meta_train_full = pd.DataFrame(columns = agg_full.columns)
    test_full = pd.DataFrame(columns = agg_full.columns)

    # filter data for respective city
    for city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:


        subset = agg_full[agg_full.assigned_city == city]

        if (target_city != city):
        # create three subsets - training data, meta_training_data, test_data
            train, meta_train = train_test_split(subset, test_size=30, random_state=41)
            # append created datasets
            train_full = train_full.append(train)
            meta_train_full = meta_train_full.append(meta_train)

        else:
            test_full = subset

    #reset index for all three subsets
    train_full = train_full.reset_index(drop = True)
    meta_train_full = meta_train_full.reset_index(drop = True)
    test_full = test_full.reset_index(drop = True)

    print('shape of training data: '+str(train_full.shape))
    print('shape of meta training data: '+str(meta_train_full.shape))
    print('shape of test data: '+str(test_full.shape))

    cities = ['marseille', 'lyon', 'paris', 'hamburg', 'bremen', 'berlin', target]
    cities.remove(target_city)

    meta_learner_train = pd.DataFrame(columns = cities)
    meta_learner_test = pd.DataFrame(columns = cities)

    # split data in feature and target
    X_meta_train_full = meta_train_full.iloc[:,2:-5]
    y_meta_train_full = meta_train_full[[target]]

    X_test_full = test_full.iloc[:,2:-5]
    y_test_full = test_full[[target]]

    for city in ['marseille', 'lyon', 'paris']:
        country = 'FR'

        if(city != target_city):
            #filter out data of respective city
            city_training_data = train_full[train_full.assigned_city == city]

            # Lasso Boosted
            #get columns lasso model selects
            cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
            # get xgboost classifier
            boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier', learner_type = 'meta')
            #make prediction for meta_train data and test data
            boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
            boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

            # PCA Lasso boosted
            # get the components pca returns
            comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
            # get columns to selct the most important components from pca
            cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
            # get fitted pca model
            pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
            #create reduced training data based on pca components
            reduced_data = pd.DataFrame(comps)
            reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
            #get trained xgboost
            boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'meta')

            #apply pca to meta_train data
            print('meta full: '+str(meta_train_full.shape))
            reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
            print('reduced pca: '+str(reduced_X_meta_train_full.shape))

            # apply pca to test data
            print('test full: '+str(test_full.shape))
            reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
            print('reduced pca: '+str(reduced_X_test_full.shape))

            # make predictions for meta_train data and test data
            boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
            boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

            #store predictions for meta_train in new dataframe and get the mean value of both predictions
            city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
            city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
            city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            #store predictions for test_data in new dataframe and get the mean value of both predictions
            city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
            city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
            city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            # add prediction to full data sets for meta_training and test
            meta_learner_train.loc[:,city] = city_result_meta['mean_model']
            meta_learner_test.loc[:,city] = city_result_test['mean_model']

    for city in ['bremen', 'hamburg', 'berlin']:

        if (city != target_city):

            country = 'DE'

            city_training_data = train_full[train_full.assigned_city == city]

            # Lasso Boosted
            cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
            boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier', learner_type = 'meta')
            boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
            boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

            # PCA Lasso boosted
            comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
            cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
            pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
            reduced_data = pd.DataFrame(comps)
            reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
            boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'meta')

            print('meta full: '+str(meta_train_full.shape))
            reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
            print('reduced pca: '+str(reduced_X_meta_train_full.shape))

            print('test full: '+str(test_full.shape))
            reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
            print('reduced pca: '+str(reduced_X_test_full.shape))

            boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
            boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

            city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
            city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
            city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
            city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
            city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            meta_learner_train.loc[:,city] = city_result_meta['mean_model']
            meta_learner_test.loc[:,city] = city_result_test['mean_model']   

    # add the target column to the meta_train set and test set and add cities back to dataframe    
    meta_learner_train.loc[:,target]= y_meta_train_full
    meta_learner_test.loc[:,target] = y_test_full
    meta_learner_train.loc[:, 'assigned_city'] = meta_train_full.assigned_city
    meta_learner_test.loc[:, 'assigned_city'] = test_full.assigned_city


    clf = LinearRegression()
    clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

    #make prediction for test data
    y_pred = clf.predict(meta_learner_test.iloc[:,:-2])
    naive_pred = [meta_learner_train[[target]].mean().values[0]] * len(meta_learner_test[[target]])

    meta_learner_results.loc[target_city, target] = (100-(metrics.mean_squared_error(meta_learner_test[[target]], y_pred)/metrics.mean_squared_error(meta_learner_test[[target]], naive_pred)*100))
    meta_learner_r2_score.loc[target_city, target] = metrics.r2_score(meta_learner_test[[target]], y_pred)

shape of training data: (1263, 243)
shape of meta training data: (150, 243)
shape of test data: (344, 243)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (344, 243)
reduced pca: (344, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (344, 243)
reduced pca: (344, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.5
shape before pca: (99, 236)
number of pca components: 53
shape after

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (125, 243)
reduced pca: (125, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (125, 243)
reduced pca: (125, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.5
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.5
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.5
meta full: (150, 243)
reduced pca: (150, 53)
test full: (125, 243)
reduced pca: (125, 53)
0.1


In [24]:
target = 'income_levels'

for target_city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:

    train_full = pd.DataFrame(columns = agg_full.columns)
    meta_train_full = pd.DataFrame(columns = agg_full.columns)
    test_full = pd.DataFrame(columns = agg_full.columns)

    # filter data for respective city
    for city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:


        subset = agg_full[agg_full.assigned_city == city]

        if (target_city != city):
        # create three subsets - training data, meta_training_data, test_data
            train, meta_train = train_test_split(subset, test_size=30, random_state=41)
            # append created datasets
            train_full = train_full.append(train)
            meta_train_full = meta_train_full.append(meta_train)

        else:
            test_full = subset

    #reset index for all three subsets
    train_full = train_full.reset_index(drop = True)
    meta_train_full = meta_train_full.reset_index(drop = True)
    test_full = test_full.reset_index(drop = True)

    print('shape of training data: '+str(train_full.shape))
    print('shape of meta training data: '+str(meta_train_full.shape))
    print('shape of test data: '+str(test_full.shape))

    cities = ['marseille', 'lyon', 'paris', 'hamburg', 'bremen', 'berlin', target]
    cities.remove(target_city)

    meta_learner_train = pd.DataFrame(columns = cities)
    meta_learner_test = pd.DataFrame(columns = cities)

    # split data in feature and target
    X_meta_train_full = meta_train_full.iloc[:,2:-5]
    y_meta_train_full = meta_train_full[[target]]

    X_test_full = test_full.iloc[:,2:-5]
    y_test_full = test_full[[target]]

    for city in ['marseille', 'lyon', 'paris']:
        country = 'FR'

        if(city != target_city):
            #filter out data of respective city
            city_training_data = train_full[train_full.assigned_city == city]

            # Lasso Boosted
            #get columns lasso model selects
            cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
            # get xgboost classifier
            boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier', learner_type = 'meta')
            #make prediction for meta_train data and test data
            boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
            boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

            # PCA Lasso boosted
            # get the components pca returns
            comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'components', meta_learner=True)
            # get columns to selct the most important components from pca
            cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
            # get fitted pca model
            pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
            #create reduced training data based on pca components
            reduced_data = pd.DataFrame(comps)
            reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
            #get trained xgboost
            boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'meta')

            #apply pca to meta_train data
            print('meta full: '+str(meta_train_full.shape))
            reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
            print('reduced pca: '+str(reduced_X_meta_train_full.shape))

            # apply pca to test data
            print('test full: '+str(test_full.shape))
            reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
            print('reduced pca: '+str(reduced_X_test_full.shape))

            # make predictions for meta_train data and test data
            boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
            boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

            #store predictions for meta_train in new dataframe and get the mean value of both predictions
            city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
            city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
            city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            #store predictions for test_data in new dataframe and get the mean value of both predictions
            city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
            city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
            city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            # add prediction to full data sets for meta_training and test
            meta_learner_train.loc[:,city] = city_result_meta['mean_model']
            meta_learner_test.loc[:,city] = city_result_test['mean_model']

    for city in ['bremen', 'hamburg', 'berlin']:

        if (city != target_city):

            country = 'DE'

            city_training_data = train_full[train_full.assigned_city == city]

            # Lasso Boosted
            cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
            boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier', learner_type = 'meta')
            boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
            boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

            # PCA Lasso boosted
            comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
            cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
            pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
            reduced_data = pd.DataFrame(comps)
            reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
            boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', learner_type = 'meta')

            print('meta full: '+str(meta_train_full.shape))
            reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
            print('reduced pca: '+str(reduced_X_meta_train_full.shape))

            print('test full: '+str(test_full.shape))
            reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
            print('reduced pca: '+str(reduced_X_test_full.shape))

            boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
            boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

            city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
            city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
            city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
            city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
            city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
            city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

            meta_learner_train.loc[:,city] = city_result_meta['mean_model']
            meta_learner_test.loc[:,city] = city_result_test['mean_model']   

    # add the target column to the meta_train set and test set and add cities back to dataframe    
    meta_learner_train.loc[:,target]= y_meta_train_full
    meta_learner_test.loc[:,target] = y_test_full
    meta_learner_train.loc[:, 'assigned_city'] = meta_train_full.assigned_city
    meta_learner_test.loc[:, 'assigned_city'] = test_full.assigned_city


    clf = LinearRegression()
    clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

    #make prediction for test data
    y_pred = clf.predict(meta_learner_test.iloc[:,:-2])
    naive_pred = [meta_learner_train[[target]].mean().values[0]] * len(meta_learner_test[[target]])

    meta_learner_results.loc[target_city, target] = (100-(metrics.mean_squared_error(meta_learner_test[[target]], y_pred)/metrics.mean_squared_error(meta_learner_test[[target]], naive_pred)*100))
    meta_learner_r2_score.loc[target_city, target] = metrics.r2_score(meta_learner_test[[target]], y_pred)

shape of training data: (1263, 243)
shape of meta training data: (150, 243)
shape of test data: (344, 243)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
meta full: (150, 243)
reduced pca: (150, 66)
test full: (344, 243)
reduced pca: (344, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (344, 243)
reduced pca: (344, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
meta full: (150, 243)
reduced pca: (150, 66)
test full: (125, 243)
reduced pca: (125, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (125, 243)
reduced pca: (125, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
meta full: (150, 243)
reduced pca: (150, 53)
test full: (125, 243)
reduced pca: (125, 53)
0.1


In [34]:
meta_learner_r2_score.loc['mean'] = meta_learner_r2_score.mean()
meta_learner_r2_score

,unemployment_rate,foreign_nationals,income_levels
marseille,0.115469,0.161379,0.09823
lyon,-0.173374,-0.152299,-0.095444
paris,0.148987,-0.111399,0.134582
berlin,0.198991,-0.077528,0.126903
hamburg,0.070679,0.12844,0.06697
bremen,0.076006,-0.04257,0.139417
mean,0.072793,-0.015663,0.078443


In [41]:
meta_learner_results

,unemployment_rate,foreign_nationals,income_levels
marseille,12.189508,17.691429,10.959759
lyon,-16.79867,-14.959591,-9.343855
paris,15.476607,-9.775911,13.609479
berlin,31.147518,-7.203995,13.143534
hamburg,20.359341,12.85219,10.349731
bremen,11.396219,4.625742,18.145471


In [37]:
unemployment_transfer = loo_results_r2_score[['unemployment_rate']]
unemployment_transfer.loc[:,'leave_one_city_in_country_out'] = locico_results_r2_score[['unemployment_rate']]
unemployment_transfer.loc[:,'meta_learner'] = meta_learner_r2_score[['unemployment_rate']]
unemployment_transfer = unemployment_transfer.rename(columns = {'unemployment_rate': 'leave-one-city-out'})
unemployment_transfer.to_excel('output/results/transfer_unemployment.xlsx')
unemployment_transfer

,leave-one-city-out,leave_one_city_in_country_out,meta_learner
marseille,0.245659,0.172597,0.115469
lyon,-0.004347,0.077668,-0.173374
paris,0.127158,0.114996,0.148987
berlin,0.336146,0.324856,0.198991
hamburg,0.325853,0.338173,0.070679
bremen,0.228731,0.140561,0.076006
mean,0.209867,0.194808,0.072793


In [38]:
foreign_nationals_transfer = loo_results_r2_score[['foreign_nationals']]
foreign_nationals_transfer.loc[:,'leave_one_city_in_country_out'] = locico_results_r2_score[['foreign_nationals']]
foreign_nationals_transfer.loc[:,'meta_learner'] = meta_learner_r2_score[['foreign_nationals']]
foreign_nationals_transfer = foreign_nationals_transfer.rename(columns = {'foreign_nationals': 'leave-one-city-out'})
foreign_nationals_transfer.to_excel('output/results/transfer_foreign_nationals.xlsx')
foreign_nationals_transfer

,leave-one-city-out,leave_one_city_in_country_out,meta_learner
marseille,0.302784,-0.102659,0.161379
lyon,-0.013445,-0.015891,-0.152299
paris,0.119054,-0.03962,-0.111399
berlin,0.320267,0.239313,-0.077528
hamburg,0.19186,0.183433,0.12844
bremen,0.009735,-0.331548,-0.04257
mean,0.155042,-0.011162,-0.015663


In [40]:
income_transfer = loo_results_r2_score[['income_levels']]
income_transfer.loc[:,'leave_one_city_in_country_out'] = locico_results_r2_score[['income_levels']]
income_transfer.loc[:,'meta_learner'] = meta_learner_r2_score[['income_levels']]
income_transfer = income_transfer.rename(columns = {'income_levels': 'leave-one-city-out'})
income_transfer.to_excel('output/results/transfer_income_levels.xlsx')
income_transfer

,leave-one-city-out,leave_one_city_in_country_out,meta_learner
marseille,0.191492,0.261354,0.09823
lyon,-0.063776,0.041089,-0.095444
paris,0.099139,NaN,0.134582
berlin,0.201038,0.233561,0.126903
hamburg,0.20353,0.157895,0.06697
bremen,0.131206,0.116151,0.139417
mean,0.127105,0.16201,0.078443


### Grid search meta learner

In [29]:
out = pd.DataFrame(columns = ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen'])
target = 'income_levels'

for meta in ['reg', 'xgb', 'forest']:

    for target_city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:
        
        train_full = pd.DataFrame(columns = agg_full.columns)
        meta_train_full = pd.DataFrame(columns = agg_full.columns)
        test_full = pd.DataFrame(columns = agg_full.columns)
        
        # filter data for respective city
        for city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:
        
        
            subset = agg_full[agg_full.assigned_city == city]

            if (target_city != city):
            # create three subsets - training data, meta_training_data, test_data
                train, meta_train = train_test_split(subset, test_size=30, random_state=41)
                # append created datasets
                train_full = train_full.append(train)
                meta_train_full = meta_train_full.append(meta_train)

            else:
                test_full = subset

        #reset index for all three subsets
        train_full = train_full.reset_index(drop = True)
        meta_train_full = meta_train_full.reset_index(drop = True)
        test_full = test_full.reset_index(drop = True)

        print('shape of training data: '+str(train_full.shape))
        print('shape of meta training data: '+str(meta_train_full.shape))
        print('shape of test data: '+str(test_full.shape))

        cities = ['marseille', 'lyon', 'paris', 'hamburg', 'bremen', 'berlin', target]
        cities.remove(target_city)

        meta_learner_train = pd.DataFrame(columns = cities)
        meta_learner_test = pd.DataFrame(columns = cities)
        
        # split data in feature and target
        X_meta_train_full = meta_train_full.iloc[:,2:-5]
        y_meta_train_full = meta_train_full[[target]]

        X_test_full = test_full.iloc[:,2:-5]
        y_test_full = test_full[[target]]

        for city in ['marseille', 'lyon', 'paris']:
            country = 'FR'

            if(city != target_city):
                #filter out data of respective city
                city_training_data = train_full[train_full.assigned_city == city]

                # Lasso Boosted
                #get columns lasso model selects
                cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
                # get xgboost classifier
                boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier',meta_learner= True)
                #make prediction for meta_train data and test data
                boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
                boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

                # PCA Lasso boosted
                # get the components pca returns
                comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
                # get columns to selct the most important components from pca
                cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
                # get fitted pca model
                pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
                #create reduced training data based on pca components
                reduced_data = pd.DataFrame(comps)
                reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
                #get trained xgboost
                boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', meta_learner=True)

                #apply pca to meta_train data
                print('meta full: '+str(meta_train_full.shape))
                reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
                print('reduced pca: '+str(reduced_X_meta_train_full.shape))

                # apply pca to test data
                print('test full: '+str(test_full.shape))
                reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
                print('reduced pca: '+str(reduced_X_test_full.shape))

                # make predictions for meta_train data and test data
                boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
                boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

                #store predictions for meta_train in new dataframe and get the mean value of both predictions
                city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
                city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
                city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                #store predictions for test_data in new dataframe and get the mean value of both predictions
                city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
                city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
                city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                # add prediction to full data sets for meta_training and test
                meta_learner_train.loc[:,city] = city_result_meta['mean_model']
                meta_learner_test.loc[:,city] = city_result_test['mean_model']

        for city in ['bremen', 'hamburg', 'berlin']:

            if (city != target_city):

                country = 'DE'

                city_training_data = train_full[train_full.assigned_city == city]

                # Lasso Boosted
                cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
                boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier',meta_learner= True)
                boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
                boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

                # PCA Lasso boosted
                comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
                cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
                pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
                reduced_data = pd.DataFrame(comps)
                reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
                boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', meta_learner=True)

                print('meta full: '+str(meta_train_full.shape))
                reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
                print('reduced pca: '+str(reduced_X_meta_train_full.shape))

                print('test full: '+str(test_full.shape))
                reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
                print('reduced pca: '+str(reduced_X_test_full.shape))

                boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
                boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

                city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
                city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
                city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
                city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
                city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                meta_learner_train.loc[:,city] = city_result_meta['mean_model']
                meta_learner_test.loc[:,city] = city_result_test['mean_model']   

        # add the target column to the meta_train set and test set and add cities back to dataframe    
        meta_learner_train.loc[:,target]= y_meta_train_full
        meta_learner_test.loc[:,target] = y_test_full
        meta_learner_train.loc[:, 'assigned_city'] = meta_train_full.assigned_city
        meta_learner_test.loc[:, 'assigned_city'] = test_full.assigned_city


        if (meta == 'reg'):

            clf = LinearRegression()
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

        if (meta == 'xgb'):

            #train XGboost on meta_train data
            params = {'eta': [0.01, 0.1, 0.3, 0.5], 'max_depth': [1,2,4,8]}
            xgboost = XGBRegressor(verbosity = 0)
            clf = GridSearchCV(xgboost, params, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

        if (meta == 'forest'):

            param_grid = {'n_estimators': [10, 30, 50],'max_depth' : [1,3,5,7]}
            rfc = RandomForestRegressor(max_depth=2, random_state=41)
            clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])
            

        #make prediction for test data
        y_pred = clf.predict(meta_learner_test.iloc[:,:-2])
        naive_pred = [meta_learner_train[[target]].mean().values[0]] * len(meta_learner_test[[target]])

        out.loc[meta, target_city] = (100-(metrics.mean_squared_error(meta_learner_test[[target]], y_pred)/metrics.mean_squared_error(meta_learner_test[[target]], naive_pred)*100))

out

shape of training data: (1264, 243)
shape of meta training data: (150, 243)
shape of test data: (344, 243)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
meta full: (150, 243)
reduced pca: (150, 66)
test full: (344, 243)
reduced pca: (344, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (344, 243)
reduced pca: (344, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
meta full: (150, 243)
reduced pca: (150, 66)
test full: (126, 243)
reduced pca: (126, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (126, 243)
reduced pca: (126, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
meta full: (150, 243)
reduced pca: (150, 53)
test full: (126, 243)
reduced pca: (126, 53)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
sh

shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
meta full: (150, 243)
reduced pca: (150, 3)
test full: (861, 243)
reduced pca: (861, 3)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
meta full: (150, 243)
reduced pca: (150, 58)
test full: (861, 243)
reduced pca: (861, 58)
shape of training data: (1486, 243)
shape of meta training data: (150, 243)
shape of test data: (122, 243)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
meta 

shape of training data: (1432, 243)
shape of meta training data: (150, 243)
shape of test data: (176, 243)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
meta full: (150, 243)
reduced pca: (150, 67)
test full: (176, 243)
reduced pca: (176, 67)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (176, 243)
reduced pca: (176, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape a

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.1
meta full: (150, 243)
reduced pca: (150, 66)
test full: (129, 243)
reduced pca: (129, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (129, 243)
reduced pca: (129, 133)
0.1
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
meta full: (150, 243)
reduced pca: (150, 3)
test full: (129, 243)
reduced pca: (129, 3)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
shape befo

,marseille,lyon,paris,berlin,hamburg,bremen
reg,9.480412,2.829947,7.940443,-3.095591,12.559969,12.882937
xgb,0.728327,-2.666347,10.835621,-0.047874,-2.16654,7.759221
forest,2.49138,-1.029561,11.390178,-15.056882,6.031444,6.455641


In [30]:
out

,marseille,lyon,paris,berlin,hamburg,bremen
reg,9.480412,2.829947,7.940443,-3.095591,12.559969,12.882937
xgb,0.728327,-2.666347,10.835621,-0.047874,-2.16654,7.759221
forest,2.49138,-1.029561,11.390178,-15.056882,6.031444,6.455641


In [31]:
out = pd.DataFrame(columns = ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen'])
target = 'unemployment_rate'

for meta in ['reg', 'xgb', 'forest']:

    for target_city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:
        
        train_full = pd.DataFrame(columns = agg_full.columns)
        meta_train_full = pd.DataFrame(columns = agg_full.columns)
        test_full = pd.DataFrame(columns = agg_full.columns)
        
        # filter data for respective city
        for city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:
        
        
            subset = agg_full[agg_full.assigned_city == city]

            if (target_city != city):
            # create three subsets - training data, meta_training_data, test_data
                train, meta_train = train_test_split(subset, test_size=30, random_state=41)
                # append created datasets
                train_full = train_full.append(train)
                meta_train_full = meta_train_full.append(meta_train)

            else:
                test_full = subset

        #reset index for all three subsets
        train_full = train_full.reset_index(drop = True)
        meta_train_full = meta_train_full.reset_index(drop = True)
        test_full = test_full.reset_index(drop = True)

        print('shape of training data: '+str(train_full.shape))
        print('shape of meta training data: '+str(meta_train_full.shape))
        print('shape of test data: '+str(test_full.shape))

        cities = ['marseille', 'lyon', 'paris', 'hamburg', 'bremen', 'berlin', target]
        cities.remove(target_city)

        meta_learner_train = pd.DataFrame(columns = cities)
        meta_learner_test = pd.DataFrame(columns = cities)
        
        # split data in feature and target
        X_meta_train_full = meta_train_full.iloc[:,2:-5]
        y_meta_train_full = meta_train_full[[target]]

        X_test_full = test_full.iloc[:,2:-5]
        y_test_full = test_full[[target]]

        for city in ['marseille', 'lyon', 'paris']:
            country = 'FR'

            if(city != target_city):
                #filter out data of respective city
                city_training_data = train_full[train_full.assigned_city == city]

                # Lasso Boosted
                #get columns lasso model selects
                cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
                # get xgboost classifier
                boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier',meta_learner= True)
                #make prediction for meta_train data and test data
                boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
                boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

                # PCA Lasso boosted
                # get the components pca returns
                comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
                # get columns to selct the most important components from pca
                cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
                # get fitted pca model
                pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
                #create reduced training data based on pca components
                reduced_data = pd.DataFrame(comps)
                reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
                #get trained xgboost
                boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', meta_learner=True)

                #apply pca to meta_train data
                print('meta full: '+str(meta_train_full.shape))
                reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
                print('reduced pca: '+str(reduced_X_meta_train_full.shape))

                # apply pca to test data
                print('test full: '+str(test_full.shape))
                reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
                print('reduced pca: '+str(reduced_X_test_full.shape))

                # make predictions for meta_train data and test data
                boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
                boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

                #store predictions for meta_train in new dataframe and get the mean value of both predictions
                city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
                city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
                city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                #store predictions for test_data in new dataframe and get the mean value of both predictions
                city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
                city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
                city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                # add prediction to full data sets for meta_training and test
                meta_learner_train.loc[:,city] = city_result_meta['mean_model']
                meta_learner_test.loc[:,city] = city_result_test['mean_model']

        for city in ['bremen', 'hamburg', 'berlin']:

            if (city != target_city):

                country = 'DE'

                city_training_data = train_full[train_full.assigned_city == city]

                # Lasso Boosted
                cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
                boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier',meta_learner= True)
                boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
                boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

                # PCA Lasso boosted
                comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
                cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
                pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
                reduced_data = pd.DataFrame(comps)
                reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
                boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', meta_learner=True)

                print('meta full: '+str(meta_train_full.shape))
                reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
                print('reduced pca: '+str(reduced_X_meta_train_full.shape))

                print('test full: '+str(test_full.shape))
                reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
                print('reduced pca: '+str(reduced_X_test_full.shape))

                boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
                boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

                city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
                city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
                city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
                city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
                city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                meta_learner_train.loc[:,city] = city_result_meta['mean_model']
                meta_learner_test.loc[:,city] = city_result_test['mean_model']   

        # add the target column to the meta_train set and test set and add cities back to dataframe    
        meta_learner_train.loc[:,target]= y_meta_train_full
        meta_learner_test.loc[:,target] = y_test_full
        meta_learner_train.loc[:, 'assigned_city'] = meta_train_full.assigned_city
        meta_learner_test.loc[:, 'assigned_city'] = test_full.assigned_city


        if (meta == 'reg'):

            clf = LinearRegression()
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

        if (meta == 'xgb'):

            #train XGboost on meta_train data
            params = {'eta': [0.01, 0.1, 0.3, 0.5], 'max_depth': [1,2,3,4,8]}
            xgboost = XGBRegressor(verbosity = 0)
            clf = GridSearchCV(xgboost, params, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

        if (meta == 'forest'):

            param_grid = {'n_estimators': [10, 30, 50],'max_depth' : [1,3,5,7]}
            rfc = RandomForestRegressor(max_depth=2, random_state=41)
            clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])
            

        #make prediction for test data
        y_pred = clf.predict(meta_learner_test.iloc[:,:-2])
        naive_pred = [meta_learner_train[[target]].mean().values[0]] * len(meta_learner_test[[target]])

        out.loc[meta, target_city] = (100-(metrics.mean_squared_error(meta_learner_test[[target]], y_pred)/metrics.mean_squared_error(meta_learner_test[[target]], naive_pred)*100))

out

shape of training data: (1264, 243)
shape of meta training data: (150, 243)
shape of test data: (344, 243)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (344, 243)
reduced pca: (344, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (344, 243)
reduced pca: (344, 133)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (126, 243)
reduced pca: (126, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (126, 243)
reduced pca: (126, 133)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
meta full: (150, 243)
reduced pca: (150, 53)
test full: (126, 243)
reduced pca: (126, 53)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.2
sh

shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
meta full: (150, 243)
reduced pca: (150, 3)
test full: (861, 243)
reduced pca: (861, 3)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.2
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.2
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.2
meta full: (150, 243)
reduced pca: (150, 58)
test full: (861, 243)
reduced pca: (861, 58)
shape of training data: (1486, 243)
shape of meta training data: (150, 243)
shape of test data: (122, 243)
0.1
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.1
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.1
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.1
meta full

shape of training data: (1432, 243)
shape of meta training data: (150, 243)
shape of test data: (176, 243)
0.1
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.1
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.1
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.1
meta full: (150, 243)
reduced pca: (150, 67)
test full: (176, 243)
reduced pca: (176, 67)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (176, 243)
reduced pca: (176, 133)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (129, 243)
reduced pca: (129, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (129, 243)
reduced pca: (129, 133)
0.1
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.1
meta full: (150, 243)
reduced pca: (150, 3)
test full: (129, 243)
reduced pca: (129, 3)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.2
shape befo

,marseille,lyon,paris,berlin,hamburg,bremen
reg,3.212926,-6.972726,16.256031,30.203543,18.891194,18.9616
xgb,5.544476,-15.334352,12.841335,37.082098,12.38094,14.794661
forest,-3.457574,-8.938201,11.635354,26.591417,8.162922,4.479168


In [32]:
out

,marseille,lyon,paris,berlin,hamburg,bremen
reg,3.212926,-6.972726,16.256031,30.203543,18.891194,18.9616
xgb,5.544476,-15.334352,12.841335,37.082098,12.38094,14.794661
forest,-3.457574,-8.938201,11.635354,26.591417,8.162922,4.479168


In [33]:
out = pd.DataFrame(columns = ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen'])
target = 'foreign_nationals'

for meta in ['reg', 'xgb', 'forest']:

    for target_city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:
        
        train_full = pd.DataFrame(columns = agg_full.columns)
        meta_train_full = pd.DataFrame(columns = agg_full.columns)
        test_full = pd.DataFrame(columns = agg_full.columns)
        
        # filter data for respective city
        for city in ['marseille', 'lyon', 'paris', 'berlin', 'hamburg', 'bremen']:
        
        
            subset = agg_full[agg_full.assigned_city == city]

            if (target_city != city):
            # create three subsets - training data, meta_training_data, test_data
                train, meta_train = train_test_split(subset, test_size=30, random_state=41)
                # append created datasets
                train_full = train_full.append(train)
                meta_train_full = meta_train_full.append(meta_train)

            else:
                test_full = subset

        #reset index for all three subsets
        train_full = train_full.reset_index(drop = True)
        meta_train_full = meta_train_full.reset_index(drop = True)
        test_full = test_full.reset_index(drop = True)

        print('shape of training data: '+str(train_full.shape))
        print('shape of meta training data: '+str(meta_train_full.shape))
        print('shape of test data: '+str(test_full.shape))

        cities = ['marseille', 'lyon', 'paris', 'hamburg', 'bremen', 'berlin', target]
        cities.remove(target_city)

        meta_learner_train = pd.DataFrame(columns = cities)
        meta_learner_test = pd.DataFrame(columns = cities)
        
        # split data in feature and target
        X_meta_train_full = meta_train_full.iloc[:,2:-5]
        y_meta_train_full = meta_train_full[[target]]

        X_test_full = test_full.iloc[:,2:-5]
        y_test_full = test_full[[target]]

        for city in ['marseille', 'lyon', 'paris']:
            country = 'FR'

            if(city != target_city):
                #filter out data of respective city
                city_training_data = train_full[train_full.assigned_city == city]

                # Lasso Boosted
                #get columns lasso model selects
                cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
                # get xgboost classifier
                boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier',meta_learner= True)
                #make prediction for meta_train data and test data
                boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
                boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

                # PCA Lasso boosted
                # get the components pca returns
                comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
                # get columns to selct the most important components from pca
                cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
                # get fitted pca model
                pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
                #create reduced training data based on pca components
                reduced_data = pd.DataFrame(comps)
                reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
                #get trained xgboost
                boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', meta_learner=True)

                #apply pca to meta_train data
                print('meta full: '+str(meta_train_full.shape))
                reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
                print('reduced pca: '+str(reduced_X_meta_train_full.shape))

                # apply pca to test data
                print('test full: '+str(test_full.shape))
                reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
                print('reduced pca: '+str(reduced_X_test_full.shape))

                # make predictions for meta_train data and test data
                boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
                boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

                #store predictions for meta_train in new dataframe and get the mean value of both predictions
                city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
                city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
                city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                #store predictions for test_data in new dataframe and get the mean value of both predictions
                city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
                city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
                city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                # add prediction to full data sets for meta_training and test
                meta_learner_train.loc[:,city] = city_result_meta['mean_model']
                meta_learner_test.loc[:,city] = city_result_test['mean_model']

        for city in ['bremen', 'hamburg', 'berlin']:

            if (city != target_city):

                country = 'DE'

                city_training_data = train_full[train_full.assigned_city == city]

                # Lasso Boosted
                cols_lasso = get_best_lasso_model(agg=city_training_data, target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output ='used_columns',meta_learner= True)
                boosted_lasso = train_xgboost(city_training_data, target, cols_lasso, 'classifier',meta_learner= True)
                boosted_lasso_predicts_meta = boosted_lasso.predict(X_meta_train_full[cols_lasso])
                boosted_lasso_predicts_test = boosted_lasso.predict(X_test_full[cols_lasso])

                # PCA Lasso boosted
                comps = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,  density_type='count', radius = 1000, output = 'components', meta_learner=True)
                cols_pca_lasso = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015, density_type='count', radius = 1000, output = 'used_columns', meta_learner=True)
                pca = get_best_pca_lasso_model(agg=city_training_data,target=target, city=city, country=country, socio_year=2015,density_type='count', radius = 1000, output = 'pca_classifier', meta_learner=True)
                reduced_data = pd.DataFrame(comps)
                reduced_data = reduced_data.join(city_training_data.iloc[:,-5:].reset_index(drop = True))
                boosted_pca_lasso = train_xgboost(reduced_data, target, cols_pca_lasso, 'classifier', meta_learner=True)

                print('meta full: '+str(meta_train_full.shape))
                reduced_X_meta_train_full = pd.DataFrame(pca.transform(X_meta_train_full))
                print('reduced pca: '+str(reduced_X_meta_train_full.shape))

                print('test full: '+str(test_full.shape))
                reduced_X_test_full = pd.DataFrame(pca.transform(X_test_full))
                print('reduced pca: '+str(reduced_X_test_full.shape))

                boosted_pca_lasso_predicts_meta = boosted_pca_lasso.predict(reduced_X_meta_train_full[cols_pca_lasso])
                boosted_pca_lasso_predicts_test = boosted_pca_lasso.predict(reduced_X_test_full[cols_pca_lasso])

                city_result_meta = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_meta.loc[:,'boosted_lasso'] = boosted_lasso_predicts_meta
                city_result_meta.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_meta
                city_result_meta.loc[:, 'mean_model'] = city_result_meta[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                city_result_test = pd.DataFrame(columns = ['boosted_lasso', 'boosted_pca_lasso'])
                city_result_test.loc[:,'boosted_lasso'] = boosted_lasso_predicts_test
                city_result_test.loc[:,'boosted_pca_lasso'] = boosted_pca_lasso_predicts_test
                city_result_test.loc[:, 'mean_model'] = city_result_test[['boosted_lasso', 'boosted_pca_lasso']].mean(axis = 1)

                meta_learner_train.loc[:,city] = city_result_meta['mean_model']
                meta_learner_test.loc[:,city] = city_result_test['mean_model']   

        # add the target column to the meta_train set and test set and add cities back to dataframe    
        meta_learner_train.loc[:,target]= y_meta_train_full
        meta_learner_test.loc[:,target] = y_test_full
        meta_learner_train.loc[:, 'assigned_city'] = meta_train_full.assigned_city
        meta_learner_test.loc[:, 'assigned_city'] = test_full.assigned_city


        if (meta == 'reg'):

            clf = LinearRegression()
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

        if (meta == 'xgb'):

            #train XGboost on meta_train data
            params = {'eta': [0.01, 0.1, 0.3, 0.5], 'max_depth': [1,2,3,4,8]}
            xgboost = XGBRegressor(verbosity = 0)
            clf = GridSearchCV(xgboost, params, scoring = ['neg_mean_squared_error'], refit ='neg_mean_squared_error')
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])

        if (meta == 'forest'):

            param_grid = {'n_estimators': [10, 30, 50],'max_depth' : [1,3,5,7]}
            rfc = RandomForestRegressor(max_depth=2, random_state=41)
            clf = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
            clf.fit(meta_learner_train.iloc[:,:-2],meta_learner_train.loc[:,[target]])
            

        #make prediction for test data
        y_pred = clf.predict(meta_learner_test.iloc[:,:-2])
        naive_pred = [meta_learner_train[[target]].mean().values[0]] * len(meta_learner_test[[target]])

        out.loc[meta, target_city] = (100-(metrics.mean_squared_error(meta_learner_test[[target]], y_pred)/metrics.mean_squared_error(meta_learner_test[[target]], naive_pred)*100))

out

shape of training data: (1264, 243)
shape of meta training data: (150, 243)
shape of test data: (344, 243)
0.1
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (344, 243)
reduced pca: (344, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (344, 243)
reduced pca: (344, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (126, 243)
reduced pca: (126, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (126, 243)
reduced pca: (126, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
meta full: (150, 243)
reduced pca: (150, 53)
test full: (126, 243)
reduced pca: (126, 53)
0.01
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
s

shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.2
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.2
meta full: (150, 243)
reduced pca: (150, 3)
test full: (861, 243)
reduced pca: (861, 3)
0.01
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
meta full: (150, 243)
reduced pca: (150, 58)
test full: (861, 243)
reduced pca: (861, 58)
shape of training data: (1486, 243)
shape of meta training data: (150, 243)
shape of test data: (122, 243)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
meta

shape of training data: (1432, 243)
shape of meta training data: (150, 243)
shape of test data: (176, 243)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
shape before pca: (314, 236)
number of pca components: 67
shape after pca: (314, 67)
0.01
meta full: (150, 243)
reduced pca: (150, 67)
test full: (176, 243)
reduced pca: (176, 67)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (176, 243)
reduced pca: (176, 133)
0.1
shape before pca: (99, 236)
number of pca components: 53
shape after pca: (99, 53)
0.2
shape before pca: (99, 236)
number of pca components: 53
shape a

shape before pca: (146, 236)
number of pca components: 66
shape after pca: (146, 66)
0.2
meta full: (150, 243)
reduced pca: (150, 66)
test full: (129, 243)
reduced pca: (129, 66)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
shape before pca: (831, 236)
number of pca components: 133
shape after pca: (831, 133)
0.01
meta full: (150, 243)
reduced pca: (150, 133)
test full: (129, 243)
reduced pca: (129, 133)
0.6
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.2
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.2
shape before pca: (96, 236)
number of pca components: 3
shape after pca: (96, 3)
0.2
meta full: (150, 243)
reduced pca: (150, 3)
test full: (129, 243)
reduced pca: (129, 3)
0.01
shape before pca: (92, 236)
number of pca components: 58
shape after pca: (92, 58)
0.1
shape bef

,marseille,lyon,paris,berlin,hamburg,bremen
reg,17.278551,-11.782764,-10.262545,-1.347399,8.039799,12.583923
xgb,4.679543,-6.318706,-10.386546,12.187936,3.535537,-1.620316
forest,1.119214,-15.460914,-10.845762,10.628664,0.513415,-1.217524


In [34]:
out

,marseille,lyon,paris,berlin,hamburg,bremen
reg,17.278551,-11.782764,-10.262545,-1.347399,8.039799,12.583923
xgb,4.679543,-6.318706,-10.386546,12.187936,3.535537,-1.620316
forest,1.119214,-15.460914,-10.845762,10.628664,0.513415,-1.217524
